In [1]:
import json
import numpy as np
import os
import pandas as pd
import urllib2

In [29]:
# coin = 'BTC'
# df_list=[]
# url = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_'+coin+'&start=1439014500&end=9999999999&period=300'
# openUrl = urllib2.urlopen(url)
# r = openUrl.read()
# openUrl.close()
# d = json.loads(r.decode())
# print pd.DataFrame(d).shape

(253208, 8)
(253208, 8)
(253208, 8)
(253208, 8)


In [2]:
url = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1356998100&end=9999999999&period=300'
openUrl = urllib2.urlopen(url)
r = openUrl.read()
openUrl.close()
d = json.loads(r.decode())

In [9]:
df = pd.DataFrame(d)
original_columns=[u'close', u'date', u'high', u'low', u'open']
df = df.loc[:,original_columns]
df.head()

,close,date,high,low,open
0,225.0,1424373000,0.33,225.0,0.33
1,225.0,1424373300,225.00,225.0,225.00
2,225.0,1424373600,225.00,225.0,225.00
3,225.0,1424373900,225.00,225.0,225.00
4,225.0,1424374200,225.00,225.0,225.00


In [10]:
df = df.set_index('date')
df.head()

,close,high,low,open
date,,,,
1424373000,225.0,0.33,225.0,0.33
1424373300,225.0,225.00,225.0,225.00
1424373600,225.0,225.00,225.0,225.00
1424373900,225.0,225.00,225.0,225.00
1424374200,225.0,225.00,225.0,225.00


In [11]:
df.to_csv('data/historical_bitcoin.csv')

In [17]:
import h5py
with h5py.File(''.join(['data/bitcoin2015to2017_wf.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value
    input_times = hf['input_times'].value
    output_times = hf['output_times'].value
    original_inputs = hf['original_inputs'].value
    original_outputs = hf['original_outputs'].value
    original_datas = hf['original_datas'].value

In [ ]:
original_inputs[0].shape

In [55]:
df.to_csv('data/bitcoin2015to2017.csv',index=None)

In [31]:
import pandas as pd
pd.read_csv('data/historical_bitcoin.csv').head()

,date,close,high,low,open
0,1424373000,225.0,0.33,225.0,0.33
1,1424373300,225.0,225.00,225.0,225.00
2,1424373600,225.0,225.00,225.0,225.00
3,1424373900,225.0,225.00,225.0,225.00
4,1424374200,225.0,225.00,225.0,225.00


# Multiple Coins

In [93]:
coins = ['BTC', 'LTC', 'ETH', 'XMR']
df_list=[]
for coin in coins:
    url = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_'+coin+'&start=1424373000&end=1515315000&period=900'
    openUrl = urllib2.urlopen(url)
    r = openUrl.read()
    openUrl.close()
    d = json.loads(r.decode())
    df_list.append(pd.DataFrame(d))

In [94]:
date_min = None
date_max = None
for df in df_list:
    if date_min == None:
        date_min = df.date.min()
        date_max = df.date.max()
    else:
        if df.date.max()< date_max :
            date_max = df.date.max()
        if df.date.min() > date_min:
            date_min = df.date.min()
date_min, date_max

(1439014500, 1515314700)

In [95]:
for i in range(len(df_list)):
    date_mask = (df_list[i].date >= date_min) & (df_list[i].date <= date_max)
    times = df_list[i].loc[date_mask].date
    df_list[i] = df_list[i].loc[date_mask,[ u'close',u'high', u'low', u'open']]

In [96]:
for df in df_list:
    print df.shape
print times.shape

(84779, 4)
(84779, 4)
(84779, 4)
(84779, 4)
(84779,)


In [97]:
## pickle is so good~
import cPickle as pickle
with open('data/allcoins.p', 'wb') as fp:
    pickle.dump(df_list, fp)

In [98]:
with open('data/times.p', 'wb') as fp:
    pickle.dump(dates, fp)

In [71]:
with open('data/allcoins.p', 'rb') as fp:
    dfl = pickle.load(fp)

In [75]:
dfl = [ np.expand_dims(df, 1)for df in dfl]
np.hstack(dfl).shape

(84779, 4, 5)

In [74]:
dfl[0].shape

(84779, 5)